In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML, display
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.api import OLS
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score
from pandas.plotting import scatter_matrix

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


### Read in and basic overview

In [4]:
path_dataset = 'gdrive/My Drive/CS109BProject/data/Capital_Projects.csv'
df = pd.read_csv(path_dataset)
print(df.shape)
df.tail()

(2259, 16)


,Date Reported As Of,PID,Project Name,Description,Category,Borough,Managing Agency,Client Agency,Current Phase,Design Start,Budget Forecast,Latest Budget Changes,Total Budget Changes,Forecast Completion,Latest Schedule Changes,Total Schedule Changes
2254,09/01/2019 12:00:00 AM,1010,PRIOR NOTICE SIDEWALKS - BROOKLYN,PRIOR NOTICE SIDEWALKS - BROOKLYN,Streets and Roadways,NaN,DDC,DOT,Construction,08/06/2015,29427000.0,NaN,NaN,05/13/2020,NaN,NaN
2255,09/01/2019 12:00:00 AM,1011,"RECONSTRUCTION OF ROSEDALE AREA, PHASE 2",NaN,Streets and Roadways,NaN,DDC,DOT,Design,06/07/2018,39360000.0,NaN,NaN,08/14/2024,NaN,NaN
2256,09/01/2019 12:00:00 AM,1012,"RECONSTRUCTION OF FRONT STREET, MANHATTAN","Reconstruction of Front Street, Manhattan",Streets and Roadways,NaN,DDC,DOT,Construction Procurement,07/31/2015,27356000.0,NaN,NaN,06/16/2022,NaN,NaN
2257,09/01/2019 12:00:00 AM,1013,"GRAND CONCOURSE, PHASE 5",THE FIFTH PHASE OF THE RECONSTRUCTION OF THE G...,Streets and Roadways,NaN,DDC,DOT,Design,09/16/2019,78921000.0,NaN,NaN,09/13/2025,NaN,NaN
2258,09/01/2019 12:00:00 AM,1014,RECONSTRUCTION OF BROADWAY PHASE 2,RECONSTRUCTION OF BROADWAY PHASE 2 - The propo...,Streets and Roadways,NaN,DDC,DOT,Design,08/22/2019,31110000.0,NaN,NaN,07/17/2026,NaN,NaN


In [0]:
df.describe()

,PID,Budget Forecast,Latest Budget Changes,Total Budget Changes,Latest Schedule Changes,Total Schedule Changes
count,2259.000000,2.153000e+03,1.881000e+03,2.194000e+03,1866.000000,2186.000000
mean,487.699867,1.456504e+08,2.342525e+06,1.838732e+07,74.543944,669.718207
std,258.005332,3.241323e+08,3.387314e+07,7.440035e+07,1115.490817,718.792415
min,3.000000,-4.102606e+06,-3.129920e+08,-2.919002e+08,-32620.000000,-1462.000000
25%,314.000000,3.913500e+07,-1.750000e+05,0.000000e+00,0.000000,77.000000
50%,504.000000,6.078000e+07,1.300000e+04,5.827000e+06,0.000000,621.000000
75%,668.000000,1.103256e+08,2.100000e+06,2.990000e+07,149.500000,1097.000000
max,1014.000000,2.631661e+09,7.288500e+08,8.078026e+08,32872.000000,3253.000000


In [5]:
df.describe(include = 'all')

# Comparing the 1st row we see that there seem to be missing values in all fields except for
# date reported, PID, Project Name, Category and Managing Agency

,Date Reported As Of,PID,Project Name,Description,Category,Borough,Managing Agency,Client Agency,Current Phase,Design Start,Budget Forecast,Latest Budget Changes,Total Budget Changes,Forecast Completion,Latest Schedule Changes,Total Schedule Changes
count,2259,2259.000000,2259,2150,2259,1634,2259,1644,2154,2095,2.153000e+03,1.881000e+03,2.194000e+03,2137,1866.000000,2186.000000
unique,15,NaN,377,530,22,19,12,30,6,341,NaN,NaN,NaN,1019,NaN,NaN
top,09/01/2019 12:00:00 AM,NaN,Macombs Dam Bridge Rehabilitation,Design and construction of a new school,Streets and Roadways,Brooklyn,DDC,DEP,Design,06/30/2004,NaN,NaN,NaN,09/03/2021,NaN,NaN
freq,378,NaN,15,114,483,455,901,784,932,42,NaN,NaN,NaN,58,NaN,NaN
mean,NaN,487.699867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.456504e+08,2.342525e+06,1.838732e+07,NaN,74.543944,669.718207
std,NaN,258.005332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.241323e+08,3.387314e+07,7.440035e+07,NaN,1115.490817,718.792415
min,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.102606e+06,-3.129920e+08,-2.919002e+08,NaN,-32620.000000,-1462.000000
25%,NaN,314.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.913500e+07,-1.750000e+05,0.000000e+00,NaN,0.000000,77.000000
50%,NaN,504.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.078000e+07,1.300000e+04,5.827000e+06,NaN,0.000000,621.000000
75%,NaN,668.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.103256e+08,2.100000e+06,2.990000e+07,NaN,149.500000,1097.000000


In [6]:
#df = df.convert_objects(convert_numeric=True)
df.dtypes

#pids = df.groupby('PID')
#pids.describe()
#df['Total Schedule Changes'] = df['Total Schedule Changes'].astype(float)
df.dtypes

Date Reported As Of         object
PID                          int64
Project Name                object
Description                 object
Category                    object
Borough                     object
Managing Agency             object
Client Agency               object
Current Phase               object
Design Start                object
Budget Forecast            float64
Latest Budget Changes      float64
Total Budget Changes       float64
Forecast Completion         object
Latest Schedule Changes    float64
Total Schedule Changes     float64
dtype: object

## Regression Model

In [7]:
df['Forecast Completion'] =pd.to_datetime(df['Forecast Completion'])
df['Date Reported As Of'] = pd.to_datetime(df['Date Reported As Of'])
df['Design Start'] = pd.to_datetime(df['Design Start'])
df.dtypes

Date Reported As Of        datetime64[ns]
PID                                 int64
Project Name                       object
Description                        object
Category                           object
Borough                            object
Managing Agency                    object
Client Agency                      object
Current Phase                      object
Design Start               datetime64[ns]
Budget Forecast                   float64
Latest Budget Changes             float64
Total Budget Changes              float64
Forecast Completion        datetime64[ns]
Latest Schedule Changes           float64
Total Schedule Changes            float64
dtype: object

In [8]:
df['Month'] = df['Date Reported As Of'].dt.month
df['Year'] = df['Date Reported As Of'].dt.year
df.head()

,Date Reported As Of,PID,Project Name,Description,Category,Borough,Managing Agency,Client Agency,Current Phase,Design Start,Budget Forecast,Latest Budget Changes,Total Budget Changes,Forecast Completion,Latest Schedule Changes,Total Schedule Changes,Month,Year
0,2014-05-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Design,2013-09-23,1.829810e+08,-1915400.0,-4318643.37,2020-01-14,1.0,270.0,5,2014
1,2015-02-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction Procurement,2013-09-23,1.687518e+08,-14229191.0,-4318643.37,2020-07-19,187.0,270.0,2,2015
2,2015-08-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction Procurement,2013-09-23,1.655324e+08,-3219322.0,-4318643.37,2020-08-08,20.0,270.0,8,2015
3,2016-01-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.692764e+08,3743944.0,-4318643.37,2020-04-01,-129.0,270.0,1,2016
4,2016-06-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.692764e+08,2.0,-4318643.37,2020-04-13,12.0,270.0,6,2016


In [9]:
df['Final Date'] = df.groupby(['PID'])['Forecast Completion'].transform('last')
df.head(15)

,Date Reported As Of,PID,Project Name,Description,Category,Borough,Managing Agency,Client Agency,Current Phase,Design Start,Budget Forecast,Latest Budget Changes,Total Budget Changes,Forecast Completion,Latest Schedule Changes,Total Schedule Changes,Month,Year,Final Date
0,2014-05-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Design,2013-09-23,1.829810e+08,-1915400.0,-4318643.37,2020-01-14,1.0,270.0,5,2014,2020-10-09
1,2015-02-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction Procurement,2013-09-23,1.687518e+08,-14229191.0,-4318643.37,2020-07-19,187.0,270.0,2,2015,2020-10-09
2,2015-08-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction Procurement,2013-09-23,1.655324e+08,-3219322.0,-4318643.37,2020-08-08,20.0,270.0,8,2015,2020-10-09
3,2016-01-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.692764e+08,3743944.0,-4318643.37,2020-04-01,-129.0,270.0,1,2016,2020-10-09
4,2016-06-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.692764e+08,2.0,-4318643.37,2020-04-13,12.0,270.0,6,2016,2020-10-09
5,2016-11-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.696747e+08,398313.0,-4318643.37,2020-09-17,157.0,270.0,11,2016,2020-10-09
6,2017-04-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.741774e+08,4502710.0,-4318643.37,2020-11-30,74.0,270.0,4,2017,2020-10-09
7,2017-09-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.751284e+08,950994.0,-4318643.37,2020-04-28,-216.0,270.0,9,2017,2020-10-09
8,2018-03-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.751594e+08,31015.0,-4318643.37,2020-04-01,-27.0,270.0,3,2018,2020-10-09
9,2018-08-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.770740e+08,1914587.0,-4318643.37,2020-07-28,118.0,270.0,8,2018,2020-10-09


In [10]:
df['Stage'] = (df['Date Reported As Of'] - df['Design Start']) / (df['Final Date']- df['Design Start'])
df.head(15)

,Date Reported As Of,PID,Project Name,Description,Category,Borough,Managing Agency,Client Agency,Current Phase,Design Start,Budget Forecast,Latest Budget Changes,Total Budget Changes,Forecast Completion,Latest Schedule Changes,Total Schedule Changes,Month,Year,Final Date,Stage
0,2014-05-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Design,2013-09-23,1.829810e+08,-1915400.0,-4318643.37,2020-01-14,1.0,270.0,5,2014,2020-10-09,0.085503
1,2015-02-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction Procurement,2013-09-23,1.687518e+08,-14229191.0,-4318643.37,2020-07-19,187.0,270.0,2,2015,2020-10-09,0.192771
2,2015-08-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction Procurement,2013-09-23,1.655324e+08,-3219322.0,-4318643.37,2020-08-08,20.0,270.0,8,2015,2020-10-09,0.263117
3,2016-01-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.692764e+08,3743944.0,-4318643.37,2020-04-01,-129.0,270.0,1,2016,2020-10-09,0.322581
4,2016-06-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.692764e+08,2.0,-4318643.37,2020-04-13,12.0,270.0,6,2016,2020-10-09,0.381656
5,2016-11-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.696747e+08,398313.0,-4318643.37,2020-09-17,157.0,270.0,11,2016,2020-10-09,0.441119
6,2017-04-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.741774e+08,4502710.0,-4318643.37,2020-11-30,74.0,270.0,4,2017,2020-10-09,0.499806
7,2017-09-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.751284e+08,950994.0,-4318643.37,2020-04-28,-216.0,270.0,9,2017,2020-10-09,0.559269
8,2018-03-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.751594e+08,31015.0,-4318643.37,2020-04-01,-27.0,270.0,3,2018,2020-10-09,0.629615
9,2018-08-01,3,26th Ward Waste Water Treatment Plant Prelimin...,The 26th Ward WWTP is mandated to be upgraded ...,Wastewater Treatment,Brooklyn,DEP,DEP,Construction,2013-09-23,1.770740e+08,1914587.0,-4318643.37,2020-07-28,118.0,270.0,8,2018,2020-10-09,0.689079


In [0]:
df.dtypes

Date Reported As Of        datetime64[ns]
PID                                 int64
Project Name                       object
Description                        object
Category                           object
Borough                            object
Managing Agency                    object
Client Agency                      object
Current Phase                      object
Design Start               datetime64[ns]
Budget Forecast                   float64
Latest Budget Changes             float64
Total Budget Changes              float64
Forecast Completion        datetime64[ns]
Latest Schedule Changes           float64
Total Schedule Changes            float64
Month                               int64
Year                                int64
Final Date                 datetime64[ns]
Stage                             float64
dtype: object

In [41]:
categorical_columns = ['Category', 'Borough', 'Current Phase'] # not including Agencies nor Decription
nyc_df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)
datetime_columns = ['Design Start', 'Forecast Completion', 'Final Date']
for cols in datetime_columns:
  nyc_df[cols] = pd.to_numeric(nyc_df[cols])
nyc_df.dtypes

Date Reported As Of                       datetime64[ns]
PID                                                int64
Project Name                                      object
Description                                       object
Managing Agency                                   object
                                               ...      
Current Phase_Construction                         uint8
Current Phase_Construction Procurement             uint8
Current Phase_Design                               uint8
Current Phase_IT                                   uint8
Current Phase_Scoping/Planning                     uint8
Length: 61, dtype: object

In [42]:
nyc_df.columns

Index(['Date Reported As Of', 'PID', 'Project Name', 'Description',
       'Managing Agency', 'Client Agency', 'Design Start', 'Budget Forecast',
       'Latest Budget Changes', 'Total Budget Changes', 'Forecast Completion',
       'Latest Schedule Changes', 'Total Schedule Changes', 'Month', 'Year',
       'Final Date', 'Stage', 'Category_Bridges',
       'Category_Bridges, Streets and Roadways', 'Category_Ferries',
       'Category_Health and Hospitals', 'Category_Health and Hospitals, IT',
       'Category_IT', 'Category_Industrial Development',
       'Category_Industrial Development, Parks',
       'Category_Industrial Development, Streets and Roadways',
       'Category_Libraries', 'Category_Other Government Facilities',
       'Category_Parks', 'Category_Parks, Streets and Roadways',
       'Category_Public Safety and Criminal Justice', 'Category_Sanitation',
       'Category_Schools', 'Category_Sewers', 'Category_Social Services',
       'Category_Streets and Roadways', 'Catego

In [0]:
drop_columns = ['Date Reported As Of', 'PID', 'Project Name', 'Description','Managing Agency', 'Client Agency']
nyc_df = nyc_df.drop(drop_columns, axis=1)
nyc_df = nyc_df.dropna()

In [0]:
nyc_train, nyc_test = train_test_split(nyc_df, test_size=.2, random_state=90)

In [45]:
nyc_train.head()

,Design Start,Budget Forecast,Latest Budget Changes,Total Budget Changes,Forecast Completion,Latest Schedule Changes,Total Schedule Changes,Month,Year,Final Date,Stage,Category_Bridges,"Category_Bridges, Streets and Roadways",Category_Ferries,Category_Health and Hospitals,"Category_Health and Hospitals, IT",Category_IT,Category_Industrial Development,"Category_Industrial Development, Parks","Category_Industrial Development, Streets and Roadways",Category_Libraries,Category_Other Government Facilities,Category_Parks,"Category_Parks, Streets and Roadways",Category_Public Safety and Criminal Justice,Category_Sanitation,Category_Schools,Category_Sewers,Category_Social Services,Category_Streets and Roadways,Category_Wastewater Treatment,Category_Water Supply,"Borough_Bronx, Manhattan",Borough_Brooklyn,"Borough_Brooklyn, Queens",Borough_Carmel,Borough_Citywide,Borough_Manhattan,"Borough_Manhattan, Bronx","Borough_Manhattan, Staten Island","Borough_Manhattan, Staten Island, Bronx",Borough_Manhatten,Borough_Marlboro,Borough_New York,Borough_Olive,Borough_Port Jervis,Borough_Queens,Borough_Staten Island,Borough_Upstate,Borough_Valhalla,Current Phase_Construction,Current Phase_Construction Procurement,Current Phase_Design,Current Phase_IT,Current Phase_Scoping/Planning
2139,1563235200000000000,2.750000e+07,0.0,0.00,1688083200000000000,91.0,91.0,9,2019,1688083200000000000,0.032526,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1510,1496620800000000000,2.879600e+07,0.0,4450000.00,1591920000000000000,0.0,0.0,3,2018,1591920000000000000,0.243880,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
12,1379894400000000000,1.805777e+08,-4664980.0,-4318643.37,1602201600000000000,0.0,270.0,9,2019,1602201600000000000,0.842985,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
860,1252886400000000000,6.286300e+07,0.0,40176000.00,1571097600000000000,103.0,761.0,9,2019,1571097600000000000,0.988053,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1696,1422489600000000000,1.711353e+07,-718468.0,-16952441.80,1651968000000000000,0.0,496.0,9,2019,1651968000000000000,0.631024,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0


In [46]:
nyc_test.head()

,Design Start,Budget Forecast,Latest Budget Changes,Total Budget Changes,Forecast Completion,Latest Schedule Changes,Total Schedule Changes,Month,Year,Final Date,Stage,Category_Bridges,"Category_Bridges, Streets and Roadways",Category_Ferries,Category_Health and Hospitals,"Category_Health and Hospitals, IT",Category_IT,Category_Industrial Development,"Category_Industrial Development, Parks","Category_Industrial Development, Streets and Roadways",Category_Libraries,Category_Other Government Facilities,Category_Parks,"Category_Parks, Streets and Roadways",Category_Public Safety and Criminal Justice,Category_Sanitation,Category_Schools,Category_Sewers,Category_Social Services,Category_Streets and Roadways,Category_Wastewater Treatment,Category_Water Supply,"Borough_Bronx, Manhattan",Borough_Brooklyn,"Borough_Brooklyn, Queens",Borough_Carmel,Borough_Citywide,Borough_Manhattan,"Borough_Manhattan, Bronx","Borough_Manhattan, Staten Island","Borough_Manhattan, Staten Island, Bronx",Borough_Manhatten,Borough_Marlboro,Borough_New York,Borough_Olive,Borough_Port Jervis,Borough_Queens,Borough_Staten Island,Borough_Upstate,Borough_Valhalla,Current Phase_Construction,Current Phase_Construction Procurement,Current Phase_Design,Current Phase_IT,Current Phase_Scoping/Planning
1061,1428624000000000000,129899998.0,1998.0,29899998.0,1582848000000000000,315.0,1122.0,1,2019,1582848000000000000,0.763025,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
677,1427328000000000000,62868000.0,-54175776.0,6034000.0,1598400000000000000,0.0,730.0,5,2019,1598400000000000000,0.756061,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1980,1205193600000000000,30468274.0,4505935.0,-7853065.0,1632960000000000000,-223.0,85.0,5,2019,1632960000000000000,0.821652,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
484,1339027200000000000,114950000.0,-4967000.0,88902000.0,1717459200000000000,0.0,3253.0,5,2019,1717459200000000000,0.575114,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1578,1432944000000000000,31354000.0,930000.0,-1000.0,1624147200000000000,0.0,0.0,1,2019,1624147200000000000,0.592860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [47]:
# Budget change
cost_column = ['Latest Budget Changes']
X_train = nyc_train.drop(cost_column, axis=1)
y_train = nyc_train[cost_column[0]]
fitted_model = LinearRegression().fit(X_train, y_train)
fitted_model.score(X_train, y_train)


0.18002279199182494